# Commit Reveal Strategy
This commit reveal strategy aims to tackle unwanted weight copying behaviour bittensor. This notebook will guide you through how to run diagnostic in your SN to determine a correct parameter to set for your SN. 

## Background
Validators are encouraged to do validation work to increase competitveness in a subnet. Through reaching consensus, validators would be rewarded with dividend for their contribution in the SN. Yet, some validators choose to reach consensus through direct copying weights that was produced by other validators which hurts the decentralization characteristic of bittensor.

Commit reveal was designed such that there will be offset in time when the weights are generated by the validators and the concensus are calculated. Thus, the weights that weight copier set would always be at least `commit_reveal_weight_interval` later than the original weight. The idea is that `commit_reveal_weight_interval` should be long enough such that when the copier does the copy, there would be enough change in the network, so that the weight to copy would already be irrelevant.

Here it illustrates the difference in the existing system VS the commit reveal system. 

For the ease of illustration, assume `conceal_period = floor(commit_reveal_weight_interval / 360)`, we also suggest users to set the `commit_reveal_weight_interval` to be a multiple of 360 blocks (a tempo).

#### === Existing system ===
| Epoch  (360 blocks = 1 tempo apart)    | Actor |Actions                                                                                         |
|-----------|--------|-----------------------------------------------------------------------------------------|
| n     | Validators |Does evaluation and set weight to the chain.                                          |
| n     | Weight copier | Weights would be available on the chain, weight copier can copy weights available. |
| n | Chain |Calculates consensus based on validator weights.                                           |
| n | Weight copier|Consensus would be available on the chain, weight copier can copy consensus available.   |

#### === Commit reveal system ===
| Epoch (360 blocks = 1 tempo apart)              | Actor | Action                                                                                                                       |
|--------------------|-----------------|---------------------------------------------------------------------------------------------------------|
| n - `conceal_period` | Validators | Does evaluation and set hashed weight hash(hotkey, weight_old) to the chain.                                      |
| n                  | Validators | Set weight to the chain that corresponds to hash(hotkey, weight_old).                                             |
| n                  | Weight copier | Weights would be available on the chain, weight copier can copy weights available and set hash(hotkey, weight_old) to chain. |
| n                  | Chain |Calculates consensus based on hashes received on n - `conceal_period`.                                               |
| n                  | Weight copier | Consensus would be available on the chain, weight copier can copy consensus available and set them as hash(hotkey, weight_old) to chain. |

* notice how when weight copier set weights, the weight it set is already `concel_period` apart from when the weight was generated. 


Here is the result from a recent run dated from block 4766968.
`cr` indicates the suggested commit reveal interval to set for each subnet.
Please do not forget to increase the immunity period by the amount of cr interval as well.
For more details of the simulation result, you can refer to the summary.html. 

|   netuid |cr    |discount in weight copier dividend |
|---------:|-----:|-----------------------:|
|        0 |    0 |            nan         |
|        1 |   16 |              0.835989  |
|        2 |    0 |            nan         |
|        3 |   24 |              0.453368  |
|        4 |    4 |              0.959481  |
|        5 |    4 |              0.931896  |
|        6 |   24 |              0.787124  |
|        7 |    0 |            nan         |
|        8 |   16 |            nan         |
|        9 |   24 |            nan         |
|       10 |   16 |              0.814848  |
|       11 |   24 |              0.163019  |
|       12 |   12 |            nan         |
|       13 |    0 |            nan         |
|       15 |   16 |              0.81722   |
|       16 |   20 |              0.802289  |
|       17 |    0 |            nan         |
|       19 |   16 |              0.896415  |
|       20 |   16 |              0.932563  |
|       24 |    0 |            nan         |
|       25 |   24 |              0.811225  |
|       27 |   24 |              0.802581  |
|       29 |   16 |              0.593398  |
|       30 |   24 |              0.801993  |
|       31 |   24 |              0.741984  |
|       32 |    0 |            nan         |
|       33 |    0 |            nan         |
|       34 |    0 |            nan         |
|       37 |   24 |              0.0740674 |
|       39 |   24 |              0.426801  |
|       40 |   20 |              0.836582  |
|       41 |   12 |              0.710663  |
|       44 |   24 |            nan         |
|       45 |   16 |              0.891898  |
|       46 |    4 |              0.965035  |
|       47 |   24 |              0.740649  |
|       48 |   24 |              0.589539  |
|       49 |   12 |              0.934864  |
|       50 |    0 |            nan         |
|       52 |   12 |              0.913682  |
|       53 |   24 |              0.348945  |
|       54 |   12 |            nan         |
|       55 |    0 |            nan         |
|       56 |   24 |              0.901693  |
|       57 |   12 |              0.788009  |
|       58 |    0 |            nan         |

In [ ]:
from IPython.display import HTML
HTML(filename='./summary.html')

## Disanostic

### Imports & setup

Make sure the required packages are installed in `requirement.txt`

`pip install -r .requirements.txt`

Note that the setup below will take very long to run the simulation, it toke me 20 mins to comeple my simulation with a machine with 32 threads.  
You may speed it up in a couple ways. 

- lower the data_points to `data_points = 100`, the default `data_points = 300` means taking data_points * 360 / 7200 = 15 days of data to generate the result.

- set the cr_interval to `cr_intervals = list(range(30, 5))`, the list of cr_interval here refers to which cr_interval will be tested on. 

- set the number of processes `processes` to match with the number of threads in your machine.

In [ ]:
import os 
import pandas as pd
import pickle
from experiment_setup import ExperimentSetup
import plotly.express as px

end_block = 4766968 # set it to the more recent subtensor block
data_points = 200
tempo = 360

setup = ExperimentSetup(
    netuids = [1],
    start_block = end_block - data_points * tempo,
    data_points = data_points,
    processes = 30,
    cr_intervals = list(range(0, 25, 4)), 
    log = True
)

print(setup)

### Download metagraphs

In [26]:
from download_metagraphs import DownloadMetagraph
DownloadMetagraph(setup = setup).run()

### Run simulation

In [ ]:
from weight_copy_simulation import WeightCopySimulation
WeightCopySimulation(setup = setup).run_simulation()

### Analysis

#### Getting relative dividend rate

With $D$ as dividend; $S$ as stake; $\mathcal Z$ as the set of validators.
We use the relative dividend rate of the copier $j$,
$$G^j = \frac{D^j/S^j}{\underset{i \in \mathcal Z \setminus \{j\}}{\mathrm{median}} \{D^i/S^i\}}$$
to measure the success of the commit-reveal approach. Here, validator dividend is normalized by the corresponding validator stake as dividend is linear in the amount of stake. Further, we use median as the baseline for comparison.

In [2]:
div_losts = {}
yuma_results = {}

for netuid in setup.netuids:
    div_losts[netuid] = {}
    yuma_results[netuid] = {}
    
    for cr_interval in setup.cr_intervals:
        file_name = f"{setup.result_path}/yuma_result_netuid{netuid}_conceal{cr_interval}.pkl"

        if not os.path.isfile(file_name):
            continue
    
        with open(file_name, 'rb') as handle:
            _yuma_results = pickle.load(handle)

        dividend = [
            (s["validator_reward_normalized"] / s["stake"]).tolist()
            for idx, s in _yuma_results.items()
        ]

        dividend_df = pd.DataFrame(
            dividend,
            columns=[f"v{i}" for i in range(len(dividend[0]) - 1 )] + ["v_bad"],
            index = _yuma_results.keys()
        )
        
        div_last = dividend_df.iloc[-1]
        div_lost = div_last.iloc[-1] / div_last.iloc[:-1].median()

        div_losts[netuid][cr_interval] = div_lost
        yuma_results[netuid][cr_interval] = _yuma_results

div_losts = pd.DataFrame(div_losts, dtype='float64')# index as commit reveal weight interval 
div_losts

/home/isabella/opentensor/research/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/isabella/opentensor/research/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/isabella/opentensor/research/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/isabella/opentensor/research/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/isabella/opentensor/research/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepd

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,52,53,54,55,56,57,58
0,5.231929,1.227412,1.017026,4.695171,1.078913,1.020959,1.269019,1.007023,1.017763,2.102466,...,1.128726,1.120041,NaN,1.122698,3.420564,0.953554,0.937300,1.096342,1.347060,0.993799
4,5.239451,1.143721,1.002073,4.829127,1.020624,0.928251,1.106519,1.078912,1.019406,1.721814,...,1.018854,1.107468,NaN,1.102389,2.544569,1.693307,1.010026,1.129413,1.177029,0.999806
8,5.231626,1.078365,0.996323,3.864723,1.010072,0.964469,1.452172,1.083444,1.094595,1.749274,...,0.934103,1.060142,NaN,1.041702,1.904377,1.391149,1.060656,1.072546,1.095356,0.975978
12,5.226816,1.064561,1.004476,2.318666,1.016941,0.983521,1.265442,1.054349,1.040992,14.761468,...,0.822616,1.033323,NaN,1.008385,1.507463,1.129226,1.014942,1.042874,1.026796,0.962271
16,5.225949,1.000636,1.010320,2.582864,1.011047,0.997161,1.082941,1.029922,1.010626,11.008026,...,0.727681,1.029385,NaN,1.005595,1.258846,1.650745,1.001600,1.042473,1.112138,0.938615
20,5.224679,1.015226,1.006322,2.412838,1.009165,1.003338,1.020017,1.029817,0.988772,5.412128,...,0.685150,1.020525,NaN,1.011582,1.332204,1.871460,1.138577,1.042376,1.177217,0.924240
24,5.217287,1.019506,1.003594,1.940332,1.011078,1.008492,0.966205,1.011614,0.978467,3.274266,...,0.640708,1.005610,NaN,1.091191,1.111116,1.995126,1.061237,0.960860,1.209205,0.910257


### Plotting the changes across commit reveal weight interval

In [ ]:
fig = px.line(
    div_losts,
    labels={
        "value": "Relative dividend rate (G)".title(),
        "index": "Commit reveal weight interval (every 360 blocks)",
        "variable": 'Subnet'
    },
    title="Relative Dividend Rate Of Weight Copier",
    width = 1000 * 1.5,
    height = 500 * 1.5,
)
fig.add_hline(y=1, line_width=3, line_dash="dash", line_color="red", annotation_text = "")
fig.update_layout(template='plotly_white')


### Conclusion 
For the commit reveal weight interval to be effective, you should set a commit reveal weight interval large enough to produce enough lost in dividend for the weight copier.

| Dividend gain (G) | Effect                                                                                                     |
|-----------------|------------------------------------------------------------------------------------------------------------|
| > 1               | There is still an advantage for weight copier to attain higher dividend than average validator. Choose the CR that gives lowest G  |
| < 1               | Norminator lost the incentive to delegate to weight copier, weight copier earn less validator take.  |
| < 0.82          | Weigh copier lost the incentive to copy weight.                                                      |

If given a commit reveal weight interval long enough (>15 hours) and the SN still fail to produce enough lost in dividend, it means that there is not enough churn and weight movement in the SN, so the existing weight copiy fix may not work for your SN. Depending on the situation, you may choose to increase competitiveness/ churn in your SN or just leave the weight copier as is. Cause when there is no churn in the SN, there would be no movement in consensus as well, so the weight copier would not be as beneficial. 

Note that when the commit reveal weight interval was set too long, it would slow down the discovery of new miners, putting them at risk for deregistration. 